In [1]:
qi("AmpelContext", "AmpelDB", "AlertProcessor", "AmpelLogger", "TarAlertLoader")

In [2]:
ctx = AmpelContext.load("ampel_conf.yaml", tier=2)
AmpelDB.delete_ampel_databases(ctx.config, "AmpelTest")
base_path = "/Users/hu/Documents/ZTF/"

In [3]:
ap = AlertProcessor(
    context = ctx,
    publish_stats = ["mongo"],
    supplier = "ZiAlertSupplier",
    log_format = "compact",
    logger_profile = "silent", # other options: 'default' or 'silent'
    verbose = 0,
    directives = {
        "channel": "VAL_TEST1",
        "stock_update": "ZiStockIngester",
        't0_add': {
            "ingester": "ZiAlertContentIngester",
            "t2_compute": {
                "ingester": "DualPointT2Ingester",
                "config": {"tags": ["ZTF"]},
                "units": [
                    {'unit': 'DemoFirstPhotoPointT2Unit'},
                    {'unit': 'DemoFirstUpperLimitT2Unit'}
                ]
            }
        }
    }
)


2020-06-03 18:22:14 AmpelDB:160 INFO
 Creating AmpelTest_data -> stock
   Creating index: {'index': [('_id', 1), ('channel', 1)], 'args': {'unique': True}}

2020-06-03 18:22:15 CORE AlertProcessor:184 INFO
 AlertProcessor setup completed


In [4]:
ap.iter_max=1
ap.set_loader(TarAlertLoader(f'{base_path}/ztf_public_20200225.tar.gz'))
ap.run()

 Creating AmpelTest_ext -> counter
 Creating AmpelTest_var -> logs
   Creating index: {'index': [('run', 1)]}
   Creating index: {'index': [('stock', 1)], 'args': {'sparse': True}}
   Creating index: {'index': [('channel', 1)], 'args': {'sparse': True}}
 Creating AmpelTest_var -> events
 Creating AmpelTest_data -> t2
   Creating index: {'index': [('stock', 1), ('channel', 1)]}
   Creating index: {'index': [('status', 1)]}
 Creating AmpelTest_data -> t0
   Creating index: {'index': [('stock', 1)]}
 Creating AmpelTest_data -> t1
   Creating index: {'index': [('stock', 1)]}


1

In [5]:
print("Datapoints in the T0 collection (1 is oldest):")
dps = sorted(ctx.db.get_collection("t0").find(), key=lambda x: x['body']['jd'], reverse=True)
for i, el in enumerate(dps):
    print(f"{len(dps)-i}: {el['_id']}")

Datapoints in the T0 collection (1 is oldest):
10: 1150106942415010004
9: 1149154782415010008
8: 1149140212415010010
7: 1146225822415010005
6: 1145226652415010008
5: -11451754868220687
4: 1142200132415010009
3: 1139178722415010006
2: 1139141562415010005
1: 1136226592415010011


In [6]:
from ampel.util.pretty import prettyjson, set_bold
t2_docs = list(ctx.db.get_collection("t2").find())
for el in t2_docs: el['_id'] = str(el['_id']) # allow serialization
print("Created T2 documents")
set_bold(prettyjson(t2_docs, indent=3), match=".*link|.*unit")

Created T2 documents


In [8]:
assert dps[-1]['_id'] == t2_docs[0]['link']